In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

In [4]:
from shared.schema import DatasetSchema

DATASET = DatasetSchema.load_schema('star-wars')
DATASET.save_schema()

In [5]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")
         .getOrCreate())

:: loading settings :: url = jar:file:/data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/egordm/.ivy2/cache
The jars for the packages stored in: /home/egordm/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa90af46-6be7-4f98-b9ec-c49c9d731bb8;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 251ms :: artifacts dl 11ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| 

# Load Graph Data

In [6]:
import json

all_edges = []
for file in DATASET.raw().glob('starwars-episode-*ns.json'):
    _, _, episode, link_type = file.stem.split('-')
    with file.open('r') as f:
        data = json.load(f)

    nodes = list(data['nodes'])
    edges = data['links']
    for e in edges:
        all_edges.append({
            'source': nodes[e['source']]['name'],
            'target': nodes[e['target']]['name'],
            'time': int(episode),
            'weight': e['value'],
            'type': link_type
        })

In [7]:
df = spark.createDataFrame(all_edges)
df.head(5)

[Row(source='CAMIE', target='LUKE', time=4, type='interactions', weight=2),
 Row(source='BIGGS', target='CAMIE', time=4, type='interactions', weight=2),
 Row(source='BIGGS', target='LUKE', time=4, type='interactions', weight=4),
 Row(source='DARTH VADER', target='LEIA', time=4, type='interactions', weight=1),
 Row(source='BERU', target='LUKE', time=4, type='interactions', weight=3)]

In [8]:
df_nodes_tmp = (
    df
        .select(F.col('source').alias('name'))
        .union(df.select(F.col('target').alias('name')))
        .distinct()
        .withColumn('id', F.monotonically_increasing_id())
)
print(df_nodes_tmp.count())
df_nodes_tmp.head(5)

113


[Row(name='C-3PO', id=0),
 Row(name='JERJERROD', id=1),
 Row(name='BERU', id=2),
 Row(name='LANDO', id=3),
 Row(name='CAMIE', id=4)]

In [9]:
df_all_edges = (
    df.join(df_nodes_tmp.alias('s'), df.source == F.col('s.name'), 'left')
        .join(df_nodes_tmp.alias('t'), df.target ==  F.col('t.name'), 'left')
        .withColumn('src', F.col('s.id'))
        .withColumn('dst', F.col('t.id'))
        .select('time', 'type', 'src', 'dst', 'weight')
        .dropDuplicates(['time', 'src', 'dst', 'type'])
)
print(df_all_edges.count())
df_all_edges.head(5)

1599


[Row(time=2, type='mentions', src=25, dst=24, weight=24),
 Row(time=2, type='mentions', src=25, dst=91, weight=1),
 Row(time=3, type='mentions', src=25, dst=11, weight=2),
 Row(time=3, type='mentions', src=39, dst=22, weight=1),
 Row(time=4, type='mentions', src=8, dst=24, weight=36)]

In [10]:
df_edges_interactions = (
    df_all_edges.filter(F.col('type') == 'interactions')
        .drop('type')
)
print(df_edges_interactions.count())
df_edges_interactions.head(5)

479


[Row(time=1, src=24, dst=45, weight=1),
 Row(time=4, src=0, dst=28, weight=2),
 Row(time=1, src=25, dst=77, weight=2),
 Row(time=2, src=36, dst=99, weight=2),
 Row(time=5, src=8, dst=105, weight=1)]

In [11]:
df_edges_mentions = (
    df_all_edges.filter(F.col('type') == 'mentions')
        .drop('type')
        .distinct()
)
print(df_edges_mentions.count())
df_edges_mentions.head(5)

1120


[Row(time=3, src=12, dst=86, weight=1),
 Row(time=7, src=47, dst=53, weight=1),
 Row(time=3, src=84, dst=27, weight=6),
 Row(time=2, src=21, dst=99, weight=2),
 Row(time=4, src=12, dst=8, weight=28)]

# WikiData Cleaning

In [12]:
from shared.constants import DATASETS_DATA_RAW

print(DATASETS_DATA_RAW.joinpath('star-wars/starwars_pages_current.xml'))

wiki_df = (
    spark.read.format('xml')
        .option("rowTag", "page")
        .load(str(DATASETS_DATA_RAW.joinpath('star-wars/starwars_pages_current.xml')))
        .select(
            'id',
            'title',
            F.col('revision.text._VALUE').alias('text'),
        )
).cache()
wiki_df.head(5)

/data/pella/projects/University/Thesis/Thesis/code/storage/datasets/raw/star-wars/starwars_pages_current.xml


[Row(id=1, title='File:Wiki.png', text='{{Top|fprot|uprot}}\n==Summary==\n{{Information\n|attention=\n|description=The logo of [[Wookieepedia]]. Cropped and modified from a picture of the [[Death Star II/Legends|second Death Star]].\n|source=Take a guess.\n|artist=*[[Tracy Duncan]] provided original design\n*[[User:Jaden Kenobi|Tyber]] provided higher-resolution version\n|filespecs=\n|licensing={{GFDL}}\n{{Cc-by-sa|3.0}}\n|other versions=[[:File:Wiki-shrinkable.png]]\n|cat artist=skip\n|cat licensee=skip\n|cat subject=skip\n|cat type=[[Category:Wookieepedia icons]]\n}}'),
 Row(id=2, title='User:LouCypher/monobook.css', text='@import url("http://www.wikicities.com/index.php?title=User:LouCypher/monobook.css&action=raw&ctype=text/css");'),
 Row(id=4, title='Template:GFDL', text='{| id="gfdl" class="darkbackground1 messagebox noprint" align="center" border="0" cellpadding="4" cellspacing="4" style="border: 1px solid #CC9; background-color: #F1F1DE"\n|-\n| [[File:Heckert_gnu.svg|70px|cente

In [13]:
wiki_chars_df = (
    wiki_df
        .filter(~F.col('title').rlike('^(Talk:|File:|User:|Forum:)'))
        .filter(F.col('text').rlike('\{\{(Character|Droid)'))
)
wiki_chars_df.head(5)

[Row(id=5, title='Brianna', text='{{Top|ffa|leg}}\n{{Youmay|the [[Echani/Legends|Echani]] [[Hybrid/Legends|hybrid]]|the [[Brianna (Human)|Human]]}}\n{{Character\n|type=Jedi\n|image=[[File:Handmaidenpromo.jpg]]\n|name=Brianna\n|homeworld=\n|birth=[[3976 BBY]]<ref name="age">{{Cite_web|url=http://forums.obsidianent.com/index.php?showtopic=25061&st=0&p=184142&#entry184142|title=Age of the Characters|date=2004-11-11|work=Obsidian Entertainment forums|archiveurl=https://web.archive.org/web/20170720114553/http://forums.obsidian.net/topic/25061-age-of-the-characters/?p=184142}}</ref>\n|death=\n|species=[[Human/Legends|Human]]/[[Echani/Legends|Echani]] [[Hybrid/Legends|hybrid]]<ref name="TSL">\'\'[[Star Wars: Knights of the Old Republic II: The Sith Lords]]\'\'</ref>\n|gender=[[Gender/Legends|Female]]<ref name="TSL" />\n|height=\n|mass=\n|hair=White<ref name="TSL" />\n|eyes=Blue<ref name="TSL" />\n|skin=\n|cyber=\n|affiliation=*[[Handmaiden Sisters]]<ref name="TSL" />\n*[[Lost Jedi]]<ref name=

In [14]:
wiki_chars_df = (
    wiki_chars_df
        .filter(~F.col('title').rlike('\/Legends$'))
        .filter(~F.col('text').rlike('^\{\{Top\|leg\}\}'))
        .filter(~F.col('text').rlike('\{\{Noncanon\|'))
        .filter(~F.col('title').rlike('lightsaber'))
        .filter(~F.col('text').rlike('\{\{Top\|.*real.*\}\}'))
        .filter(
            ~(
                F.col('text').rlike('\{\{Top\|.*leg.*\}\}') &
                ~F.col('text').rlike('\{\{Top\|.*legends=.*\}\}') &
                ~F.col('text').rlike('\{\{Top\|.*canon.*\}\}')
            )
        )
        .drop_duplicates(['id'])
).cache()
wiki_chars_df.head(5)

[Row(id=233176, title='Emergency beacon', text='{{Top|canon=Distress beacon}}\n[[File:EmergencyBeacon.jpg|thumb|right|250px|An emergency beacon]]\n{{Quote|Have you located the \'\'source\'\' of the \'\'distress beacon\'\'?\'\'"<br />"\'\'It appears to be an \'\'Imperial escape pod\'\', your \'\'Highness\'\'. \'\'One\'\' lifeform on board.|[[Leia Organa Solo|Leia Organa]] and [[Jan Dodonna/Legends|Jan Dodonna]]|Leia\'s Trust}}\nAn \'\'\'emergency beacon\'\'\' or \'\'\'distress beacon\'\'\' was a type of [[Homing beacon/Legends|homing beacon]] that was designed for use in emergency situations to summon or locate aid. Alliance operatives used them when they were in trouble, signaling other members of the Alliance during times of distress. Sometime before the [[Battle of Endor/Legends|Battle of Endor]], when [[Prince/Legends|Princess]] [[Leia Organa Solo|Leia Organa]] was preparing for a diplomatic mission to the [[Planet/Legends|planet]] of [[Yinchorr/Legends|Yinchorr]], [[Luke Skywalker/

## Data Matching

In [15]:
import unicodedata
import pandas as pd

In [16]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

db_items_df = wiki_chars_df.toPandas().set_index("id")
# db_items_df['text'] = db_items_df['text'].str.lower().apply(strip_accents)
db_items_df['title'] = db_items_df['title'].str.lower().apply(strip_accents)
db_items_df

,title,text
id,,
233176,emergency beacon,{{Top|canon=Distress beacon}}\n[[File:Emergenc...
255354,housekeeping droid,{{Top|canon=Housekeeper droid}}\n'''Housekeepi...
318541,99,{{Top|canon=Clone 99}}\n{{Character\n|type=Gal...
451699,breha organa,{{Top}}\n{{Twoconflicting|''[[Queen's Shadow]]...
453212,braylen stramm,{{Top}}\n{{Character\n|type=New Republic\n|ima...
...,...,...
683058,nan's father,{{Top}}\n{{Conjecture}}\n{{Character\n|type=Ni...
685347,ghordansk,{{Top}}\n{{Character\n|type=New Republic\n|ima...
686305,unidentified kajain'sa'nikto's ally 2,{{Top}}\n{{Conjecture}}\n{{Character\n|type=Bo...


In [17]:
VOCAB = {
    'HAN': 'Han Solo',
    'KYLO REN': 'Ben Solo',
    'REY': 'Rey Skywalker',
    'EMPEROR': 'Darh Sidious',
    'DARTH VADER': 'Anakin Skywalker',
    'FODE/BEED': 'Fodesinbeed Annodue',
    'JERJERROD': 'tiaan jerjerrod',
}

In [18]:
import Levenshtein

def distance(a, b):
    return Levenshtein.jaro_winkler(a.lower(), b.lower())

def find_match(title):
    title = title.lower().replace('count', '').replace('senator', '')\
        .replace('captain', '').replace('admiral', '').replace('general', '')\
        .replace('darth', '').replace('colonel', '').replace('clone', '')

    candidates = db_items_df[db_items_df['title'].apply(lambda x: title in x.lower())]
    if len(candidates) == 0:
        candidates = db_items_df

    # idx = candidates['title'].apply(lambda x: distance(x, title))
    # print(candidates.loc[list(idx.index)]['title'])
    # print(idx)

    idx = candidates['title'].apply(lambda x: distance(x, title)).argmax()
    match = candidates.iloc[idx]

    return match


In [19]:
names_df = df_nodes_tmp.toPandas()
node_details = []
for index, row in names_df.iterrows():
    name = row['name'] if row['name'] not in VOCAB else VOCAB[row['name']]
    match = find_match(name)
    node_details.append({
        **row.to_dict(),
        'match_title': match['title'],
        'match_id': str(match.name),
        'text': match['text'],
    })

node_names_df = pd.DataFrame(node_details).set_index('id')
node_names_df

,name,match_title,match_id,text
id,,,,
0,C-3PO,c-3po,452947,"{{Top}}\n{{Doom}}\n{{Update|[[Trouble Again]],..."
1,JERJERROD,tiaan jerjerrod,269385,{{Top}}\n{{Youmay|the [[Galactic Empire|Imperi...
2,BERU,beru whitesun lars,457342,{{Top}}\n{{Update|[[Star Wars (LINE Webtoon)]]...
3,LANDO,landonis balthazar calrissian,452964,{{Top|legends=Lando Calrissian/Legends}}\n{{Rh...
4,CAMIE,camie marstrap,512525,{{Top|legends=Camie Loneozner}}\n{{Spoiler|Cha...
...,...,...,...,...
108,TEY HOW,tey how,498826,{{Top|ga|audio=https://www.youtube.com/watch?v...
109,WALD,wald,475139,{{Top|legends=W. Wald}}\n{{Youmay|the Rodian|t...
110,VALORUM,tarsus valorum,625117,{{Top}}\n{{Character\n|type=Galactic Republic\...


## Feature Engineering

In [20]:
import wikitextparser as wtp
from lxml import html

def select_character(parsed_item):
    for template in parsed_item.templates:
        if template.name.startswith('Character') or template.name.startswith('Droid'):
            return template
    return None

def clean_value(value):
    tree = html.fromstring(f'<span>{value}</span>')
    refs = tree.find('ref')
    while refs is not None:
        refs.drop_tree()
        refs = tree.find('ref')

    value = tree.text_content().strip()

    value = wtp.remove_markup(value)

    parsed = wtp.parse(value)
    if parsed.get_lists():
        value = '|'.join(map(lambda x: x.strip(), parsed.get_lists()[0].items))

    return value.strip()

def extract_args(template):
    args = {}
    for arg in getattr(template, 'arguments', []):
        args[arg.name.strip()] = wtp.remove_markup(clean_value(arg.value.strip()))
    return args

def extract_props(text):
    parsed = wtp.parse(text)

    character = select_character(parsed)
    properties = extract_args(character)
    properties['is_droid'] = character and character.name.startswith('Droid')
    return properties

In [21]:
node_props = []
for (id, row) in node_names_df.iterrows():
    node_props.append({
        'id': id,
        **extract_props(row['text']),
    })

node_details_df = pd.DataFrame(node_props).set_index('id')
node_details_df

,image,name,homeworld,birth,death,creator,manufacturer,line,model,class,...,species,hair,eyes,skin,cyber,masters,apprentices,clan,1,2
id,,,,,,,,,,,,,,,,,,,,,
0,File:C-3PO_TLJ_Card_Trader_Award_Card.png,C-3PO,Tatooine,Prior to 32 BBY,"3 ABY, Bespin",Anakin Skywalker,Cybot Galactica,,3PO-series protocol droid,Protocol droid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,File:MoffJerjerrod-SWI187.png,Tiaan Jerjerrod,Tinnel IV,35 BBY,"4 ABY, DS-2 Death Star II Mobile Battle Statio...",NaN,NaN,NaN,NaN,NaN,...,Human,Brown,Green,Light,,,,NaN,NaN,NaN
2,File:BeruCardTrader.png,Beru Whitesun Lars,Tatooine,,"0 BBY, Tatooine",NaN,NaN,NaN,NaN,NaN,...,Human,Brown,Blue,Light,,,,NaN,NaN,NaN
3,File:LandoCalrissian-TROSOCE.png,Landonis Balthazar Calrissian,Socorro,"c. 43 BBY, Socorro",,NaN,NaN,NaN,NaN,NaN,...,Human,Black,Brown,Dark,,,,NaN,NaN,NaN
4,File:CamieMarstrap-BoBFCh2.png,Camie Marstrap,Tatooine,,,NaN,NaN,NaN,NaN,NaN,...,Human,Brown,Blue,Fair,,,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,File:Tey_How.png,Tey How,,,"32 BBY; Vuutun Palaa, Naboo system",NaN,NaN,NaN,NaN,NaN,...,Neimoidian,,,Mottled green,,,,NaN,NaN,NaN
109,File:WaldFull-SWE.png,Wald,Tatooine,38 BBY,,NaN,NaN,NaN,NaN,NaN,...,Rodian,,Black,Green,,,,NaN,NaN,NaN
110,,Tarsus Valorum,,,,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,NaN,NaN,NaN


In [22]:
import numpy as np

def strip_tokens(x):
    if not x or (isinstance(x, float) and np.isnan(x)):
        return []
    return [str(e).strip() for e in x]

node_details_df.drop(columns=['1', '2', 'clan', 'armament', 'plating', 'sensor', 'width', 'length', 'cost', 'line', 'manufacturer', 'creator', 'model', 'class', 'image'], inplace=True)
node_details_df = node_details_df.replace('', np.nan)
node_details_df['hair'] = node_details_df['hair'].str.split(',|;').apply(strip_tokens)
node_details_df['eyes'] = node_details_df['skin'].str.split(',|;').apply(strip_tokens)
node_details_df['cyber'] = node_details_df['cyber'].str.split(',|;').apply(strip_tokens)
node_details_df['skin'] = node_details_df['skin'].str.split(',|;').apply(strip_tokens)
node_details_df['masters'] = node_details_df['masters'].str.split(r'\|').apply(strip_tokens)
node_details_df['apprentices'] = node_details_df['apprentices'].str.split(r'\|').apply(strip_tokens)
node_details_df['affiliation'] = node_details_df['affiliation'].str.split(r'\|').apply(strip_tokens)
# node_details_df['height'] = node_details_df['height'].apply(lambda x: x.split()[0] if x and isinstance(x, str) else x)
# node_details_df['mass'] = node_details_df['mass'].apply(lambda x: x.split()[0] if x and isinstance(x, str) else x)
node_details_df = node_details_df.astype(object).where(pd.notnull(node_details_df), None)
node_details_df

,name,homeworld,birth,death,height,mass,gender,equipment,affiliation,is_droid,type,species,hair,eyes,skin,cyber,masters,apprentices
id,,,,,,,,,,,,,,,,,,
0,C-3PO,Tatooine,Prior to 32 BBY,"3 ABY, Bespin",1.77 meters,75 kilograms,Masculine programming,TranLang III communication module,"[Skywalker family, Confederacy of Independent ...",True,None,None,[],[],[],[],[],[]
1,Tiaan Jerjerrod,Tinnel IV,35 BBY,"4 ABY, DS-2 Death Star II Mobile Battle Statio...",1.73 meters,75 kilograms,Male,None,"[Taung & Zhell Society, Corellian Engineering ...",False,Galactic Empire,Human,[Brown],[Light],[Light],[],[],[]
2,Beru Whitesun Lars,Tatooine,None,"0 BBY, Tatooine",1.65 meters,None,Female,None,[Lars family],False,None,Human,[Brown],[Light],[Light],[],[],[]
3,Landonis Balthazar Calrissian,Socorro,"c. 43 BBY, Socorro",None,1.77 meters (5ft 10in),79 kilograms,Male,None,"[Calrissian family, Crimson Dawn, Alliance to ...",False,Resistance,Human,[Black],[Dark],[Dark],[],[],[]
4,Camie Marstrap,Tatooine,None,None,None,None,Female,None,[],False,None,Human,[Brown],[Fair],[Fair],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Tey How,None,None,"32 BBY; Vuutun Palaa, Naboo system",None,None,Female,None,[Trade Federation],False,Corporate,Neimoidian,[],[Mottled green],[Mottled green],[],[],[]
109,Wald,Tatooine,38 BBY,None,0.69 meters,None,Male,None,[],False,None,Rodian,[],[Green],[Green],[],[],[]
110,Tarsus Valorum,None,None,None,None,None,None,None,"[House Valorum, Galactic Republic]",False,Galactic Republic,None,[],[],[],[],[],[]


In [23]:
node_details_df['homeworld'] = node_details_df['homeworld'].fillna('Unknown')
node_details_df['gender'] = node_details_df['gender'].fillna('Unknown')
node_details_df['species'] = node_details_df['species'].fillna('Unknown')
node_details_df['type'] = node_details_df['type'].fillna('Unknown')
node_details_df

,name,homeworld,birth,death,height,mass,gender,equipment,affiliation,is_droid,type,species,hair,eyes,skin,cyber,masters,apprentices
id,,,,,,,,,,,,,,,,,,
0,C-3PO,Tatooine,Prior to 32 BBY,"3 ABY, Bespin",1.77 meters,75 kilograms,Masculine programming,TranLang III communication module,"[Skywalker family, Confederacy of Independent ...",True,Unknown,Unknown,[],[],[],[],[],[]
1,Tiaan Jerjerrod,Tinnel IV,35 BBY,"4 ABY, DS-2 Death Star II Mobile Battle Statio...",1.73 meters,75 kilograms,Male,None,"[Taung & Zhell Society, Corellian Engineering ...",False,Galactic Empire,Human,[Brown],[Light],[Light],[],[],[]
2,Beru Whitesun Lars,Tatooine,None,"0 BBY, Tatooine",1.65 meters,None,Female,None,[Lars family],False,Unknown,Human,[Brown],[Light],[Light],[],[],[]
3,Landonis Balthazar Calrissian,Socorro,"c. 43 BBY, Socorro",None,1.77 meters (5ft 10in),79 kilograms,Male,None,"[Calrissian family, Crimson Dawn, Alliance to ...",False,Resistance,Human,[Black],[Dark],[Dark],[],[],[]
4,Camie Marstrap,Tatooine,None,None,None,None,Female,None,[],False,Unknown,Human,[Brown],[Fair],[Fair],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Tey How,Unknown,None,"32 BBY; Vuutun Palaa, Naboo system",None,None,Female,None,[Trade Federation],False,Corporate,Neimoidian,[],[Mottled green],[Mottled green],[],[],[]
109,Wald,Tatooine,38 BBY,None,0.69 meters,None,Male,None,[],False,Unknown,Rodian,[],[Green],[Green],[],[],[]
110,Tarsus Valorum,Unknown,None,None,None,None,Unknown,None,"[House Valorum, Galactic Republic]",False,Galactic Republic,Unknown,[],[],[],[],[],[]


In [24]:
node_details_df[['affiliation']]

,affiliation
id,
0,"[Skywalker family, Confederacy of Independent ..."
1,"[Taung & Zhell Society, Corellian Engineering ..."
2,[Lars family]
3,"[Calrissian family, Crimson Dawn, Alliance to ..."
4,[]
...,...
108,[Trade Federation]
109,[]
110,"[House Valorum, Galactic Republic]"


In [25]:
from datasets.feature_transform import MultiRareLabelEncoder, MOneHotEncoder

mrare_encoder = MultiRareLabelEncoder(tol=0.05, n_categories=8)
mfeature_df = mrare_encoder.fit_transform(node_details_df[['affiliation']])

oh_encoder = MOneHotEncoder()
mfeature_df = oh_encoder.fit_transform(mfeature_df[['affiliation']]).add_prefix('feat_')

mfeature_df

,feat_affiliation_Galactic Empire,feat_affiliation_Alliance to Restore the Republic,feat_affiliation_Jedi Order,feat_affiliation_Resistance,feat_affiliation_Galactic Republic,feat_affiliation_New Republic
id,,,,,,
0,1,1,0,1,1,1
1,1,0,0,0,0,0
2,0,0,0,0,0,0
3,0,1,0,1,0,1
4,0,0,0,0,0,0
...,...,...,...,...,...,...
108,0,0,0,0,0,0
109,0,0,0,0,0,0
110,0,0,0,0,1,0


In [26]:
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder

rare_encoder = RareLabelEncoder(tol=0.05, n_categories=6)
feature_df = rare_encoder.fit_transform(node_details_df[['homeworld', 'gender', 'species', 'type']])

oh_encoder = OneHotEncoder()
feature_df = oh_encoder.fit_transform(feature_df[['homeworld', 'gender', 'species', 'type']]).add_prefix('feat_')

feature_df['feat_species_Droid'] = node_details_df['is_droid'].apply(lambda x: 1 if x else 0)
feature_df['feat_hasMaster'] = node_details_df['masters'].apply(lambda x: 1 if len(x) else 0)
feature_df['feat_hasApprentices'] = node_details_df['apprentices'].apply(lambda x: 1 if len(x) else 0)
feature_df['feat_hasCyber'] = node_details_df['cyber'].apply(lambda x: 1 if len(x) else 0)

feature_df

/data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable gender is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


,feat_homeworld_Tatooine,feat_homeworld_Rare,feat_homeworld_Naboo,feat_homeworld_Alderaan,feat_homeworld_Unknown,feat_gender_Masculine programming,feat_gender_Male,feat_gender_Female,feat_gender_Unknown,feat_gender_Feminine programming,...,feat_type_Resistance,feat_type_Rare,feat_type_Jedi,feat_type_Criminal,feat_type_Galactic Republic,feat_type_Rebel,feat_species_Droid,feat_hasMaster,feat_hasApprentices,feat_hasCyber
id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
109,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [27]:
node_details_and_features_df = node_names_df[['name']].join(feature_df).join(mfeature_df)
node_details_and_features_df.columns = [c.replace(' ', '') for c in node_details_and_features_df.columns]
node_details_and_features_df

,name,feat_homeworld_Tatooine,feat_homeworld_Rare,feat_homeworld_Naboo,feat_homeworld_Alderaan,feat_homeworld_Unknown,feat_gender_Masculineprogramming,feat_gender_Male,feat_gender_Female,feat_gender_Unknown,...,feat_species_Droid,feat_hasMaster,feat_hasApprentices,feat_hasCyber,feat_affiliation_GalacticEmpire,feat_affiliation_AlliancetoRestoretheRepublic,feat_affiliation_JediOrder,feat_affiliation_Resistance,feat_affiliation_GalacticRepublic,feat_affiliation_NewRepublic
id,,,,,,,,,,,,,,,,,,,,,
0,C-3PO,1,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,1,1,1
1,JERJERROD,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,BERU,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,LANDO,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
4,CAMIE,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,TEY HOW,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
109,WALD,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
110,VALORUM,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [28]:
df_nodes_feat = spark.createDataFrame(node_details_and_features_df.reset_index())
df_nodes_feat = (
    df_nodes_feat
        .withColumn('full_name', F.col('name'))
        .drop('name')
)
df_nodes_feat.head(5)

22/04/05 12:40:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(id=0, feat_homeworld_Tatooine=1, feat_homeworld_Rare=0, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknown=0, feat_gender_Masculineprogramming=1, feat_gender_Male=0, feat_gender_Female=0, feat_gender_Unknown=0, feat_gender_Feminineprogramming=0, feat_species_Unknown=1, feat_species_Human=0, feat_species_Rare=0, feat_type_Unknown=1, feat_type_GalacticEmpire=0, feat_type_Resistance=0, feat_type_Rare=0, feat_type_Jedi=0, feat_type_Criminal=0, feat_type_GalacticRepublic=0, feat_type_Rebel=0, feat_species_Droid=1, feat_hasMaster=0, feat_hasApprentices=0, feat_hasCyber=0, feat_affiliation_GalacticEmpire=1, feat_affiliation_AlliancetoRestoretheRepublic=1, feat_affiliation_JediOrder=0, feat_affiliation_Resistance=1, feat_affiliation_GalacticRepublic=1, feat_affiliation_NewRepublic=1, full_name='C-3PO'),
 Row(id=1, feat_homeworld_Tatooine=0, feat_homeworld_Rare=1, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknown=0, feat_gender_Masculineprogra

In [33]:
df_nodes = (
    df_nodes_tmp
        .join(df_nodes_feat, on='id', how='left')
)
df_nodes.head(5)

[Row(id=0, name='C-3PO', feat_homeworld_Tatooine=1, feat_homeworld_Rare=0, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknown=0, feat_gender_Masculineprogramming=1, feat_gender_Male=0, feat_gender_Female=0, feat_gender_Unknown=0, feat_gender_Feminineprogramming=0, feat_species_Unknown=1, feat_species_Human=0, feat_species_Rare=0, feat_type_Unknown=1, feat_type_GalacticEmpire=0, feat_type_Resistance=0, feat_type_Rare=0, feat_type_Jedi=0, feat_type_Criminal=0, feat_type_GalacticRepublic=0, feat_type_Rebel=0, feat_species_Droid=1, feat_hasMaster=0, feat_hasApprentices=0, feat_hasCyber=0, feat_affiliation_GalacticEmpire=1, feat_affiliation_AlliancetoRestoretheRepublic=1, feat_affiliation_JediOrder=0, feat_affiliation_Resistance=1, feat_affiliation_GalacticRepublic=1, feat_affiliation_NewRepublic=1, full_name='C-3PO'),
 Row(id=1, name='JERJERROD', feat_homeworld_Tatooine=0, feat_homeworld_Rare=1, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknow

In [34]:
df_nodes.write.parquet(DATASET.processed_str('nodes_Character'), mode='overwrite')

df_edges_interactions.write.parquet(DATASET.processed_str('edges_INTERACTIONS'), mode='overwrite')
df_edges_mentions.write.parquet(DATASET.processed_str('edges_MENTIONS'), mode='overwrite')

In [35]:
from shared.schema.graph import GraphSchema, NodeSchema, EdgeSchema

(
    GraphSchema()
        .add_node_schema('Character', NodeSchema.from_spark(df_nodes.schema, label='name'))
        .add_edge_schema('INTERACTIONS', EdgeSchema.from_spark(df_edges_interactions.schema, source_type='Character', target_type='Character', directed=False, timestamp='time', interaction=True))
        .add_edge_schema('MENTIONS', EdgeSchema.from_spark(df_edges_mentions.schema, source_type='Character', target_type='Character', directed=True, timestamp='time', interaction=True))
        .save_schema(DATASET.processed())
)

GraphSchema(_path=PosixPath('/data/pella/projects/University/Thesis/Thesis/code/storage/datasets/processed/star-wars'), nodes={'Character': NodeSchema(_type='Character', _schema=..., label='name', properties={'id': GraphProperty(_name='id', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'name': GraphProperty(_name='name', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False)), 'feat_homeworld_Tatooine': GraphProperty(_name='feat_homeworld_Tatooine', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'feat_homeworld_Rare': GraphProperty(_name='feat_homeworld_Rare', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'feat_homeworld_Naboo': GraphProperty(_name='feat_homeworld_Naboo', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'feat_homeworld_Alderaan': GraphProperty(_name='feat_homeworld_Alderaan', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'feat_homeworld_Unknown': GraphProperty(_name='feat_homeworld_Unknown', dty

In [36]:
df_nodes.head(5)

[Row(id=0, name='C-3PO', feat_homeworld_Tatooine=1, feat_homeworld_Rare=0, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknown=0, feat_gender_Masculineprogramming=1, feat_gender_Male=0, feat_gender_Female=0, feat_gender_Unknown=0, feat_gender_Feminineprogramming=0, feat_species_Unknown=1, feat_species_Human=0, feat_species_Rare=0, feat_type_Unknown=1, feat_type_GalacticEmpire=0, feat_type_Resistance=0, feat_type_Rare=0, feat_type_Jedi=0, feat_type_Criminal=0, feat_type_GalacticRepublic=0, feat_type_Rebel=0, feat_species_Droid=1, feat_hasMaster=0, feat_hasApprentices=0, feat_hasCyber=0, feat_affiliation_GalacticEmpire=1, feat_affiliation_AlliancetoRestoretheRepublic=1, feat_affiliation_JediOrder=0, feat_affiliation_Resistance=1, feat_affiliation_GalacticRepublic=1, feat_affiliation_NewRepublic=1, full_name='C-3PO'),
 Row(id=1, name='JERJERROD', feat_homeworld_Tatooine=0, feat_homeworld_Rare=1, feat_homeworld_Naboo=0, feat_homeworld_Alderaan=0, feat_homeworld_Unknow